In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
df = df.drop(columns = 'id')
df_test = df_test.drop(columns = 'id')

## Creatfolds.py

In [ ]:
##create folds
from sklearn import model_selection
df["kfold"] = -1

df = df.sample(frac=1).reset_index(drop=True)

kf = model_selection.StratifiedKFold(n_splits=5, shuffle=False, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X=df, y=df.claim.values)):
    print(len(train_idx), len(val_idx))
    df.loc[val_idx, 'kfold'] = fold

## Ensemble

In [ ]:
# from sklearn.model_selection import GridSearchCV
# import xgboost as xg
# from sklearn.ensemble import AdaBoostRegressor
# import xgboost as xg
# reg4 = xg.XGBClassifier(use_label_encoder=False)
# param_grid = {'tree_method' : ['gpu_hist'], 'gamma':[0.02,0.03,0.04],  'subsample':[0.6,0.7,0.8,0.9],
# 'colsample_bytree':[0.6,0.7,0.8], 'max_depth': [2,3,4]}
# grid_search = GridSearchCV(reg4,param_grid,cv = 5,scoring = 'roc_auc')

In [ ]:
# temp_X = df.drop(columns = ['kfold','claim'])
# temp_y = df.claim
# grid_search.fit(temp_X,temp_y)

In [ ]:
# grid_search.best_estimator_

### Train.py

In [ ]:
import joblib
import xgboost as xg
import pandas as pd
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
def run(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    df_train = df_train.drop(columns = 'kfold')
    df_valid = df_valid.drop(columns = 'kfold')
    x_train = df_train.drop('claim', axis=1).values
    y_train = df_train.claim.values
    x_valid = df_valid.drop('claim', axis=1).values
    y_valid = df_valid.claim.values
    clf = xg.XGBClassifier(use_label_encoder=False,
                      tree_method ='gpu_hist',
#                       scale_pos_weight=1,
#                       learning_rate=0.03,  
#                       colsample_bytree = 0.4,
#                       subsample = 0.9,
#                       objective='binary:logistic', 
#                       n_estimators=1000, 
#                       reg_alpha = 0.3,
#                       max_depth=5, 
#                       gamma=5
                          )
    clf.fit(x_train, y_train)
    y_pred = clf.predict_proba(x_valid)[:,1]
    roc_auc_score = metrics.roc_auc_score(y_valid,y_pred)
    print(f"Fold={fold}, roc_auc_score={roc_auc_score}")
    File_name = 'model' + str(fold)
    joblib.dump(
    clf,File_name)
for i in range(5):
    run(fold = i)

In [ ]:
model_0= joblib.load('./model0')
model_1 =joblib.load('./model1')
model_2= joblib.load('./model2')
model_3= joblib.load('./model3')
model_4= joblib.load('./model4')

In [ ]:
df_test

In [ ]:
y_final_2 = model_2.predict_proba(df_test)[:,1]
y_final_0 = model_0.predict_proba(df_test)[:,1]
y_final_1 = model_1.predict_proba(df_test)[:,1]
y_final_3 = model_3.predict_proba(df_test)[:,1]
y_final_4 = model_4.predict_proba(df_test)[:,1]


y_final_avg = (y_final_0+y_final_1+y_final_2+y_final_3+y_final_4)/5

In [ ]:
submission['claim'] = y_final_avg

In [ ]:
submission.to_csv('pred_csv_avg.csv',index = False)